In [33]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.applications import inception_v3
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.models import Model, load_model
from keras.layers import Dense, Dropout, BatchNormalization, ELU
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [42]:
TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'
#NUM_TRAIN, NUM_TEST = 12808, 1419
NUM_TRAIN, NUM_TEST = 34398, 3813
# Use ImageNet image sizes
IMG_WIDTH, IMG_HEIGHT = 256, 256
FC_SIZE = 1024
NUM_EPOCHS = 25
BATCH_SIZE = 54
NB_CLASSES = 18

In [40]:
inception4 = load_model("inception_v4_finetune_84.h5")

# Save the model according to the conditions
checkpoint = ModelCheckpoint("inception_v4_difftop.h5", monitor='val_acc', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
# Terminate early if validation accuracy stop improving over 10 epochs
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')

c:\users\tomforge\anaconda3\envs\aicv\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [30]:
# Initiate the train and test generators with data augumentation 
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)

# For test data, rescaling will do
test_datagen = ImageDataGenerator(
    rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = BATCH_SIZE, 
    class_mode = "categorical")

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    class_mode = "categorical")

Found 34398 images belonging to 18 classes.
Found 3813 images belonging to 18 classes.


In [38]:
for l in inception4.layers:
    l.trainable = False
x = inception4.layers[-4].output
x = Dense(512)(x)
x = BatchNormalization(name="lol")(x)
x = ELU()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
pred = Dense(NB_CLASSES, activation='softmax')(x)
new_model = Model(inputs=inception4.input, outputs=pred)
# Compile the model for training
new_model.compile(optimizer='adam', loss='categorical_hinge', metrics=['accuracy'])

In [41]:
# Train the model 
new_model.fit_generator(
    train_generator,
    steps_per_epoch = NUM_TRAIN/BATCH_SIZE,
    epochs = NUM_EPOCHS,
    validation_data = test_generator,
    callbacks = [checkpoint, early])

Epoch 1/25
238/237 [==============================] - 282s 1s/step - loss: 0.3743 - acc: 0.8051 - val_loss: 0.3969 - val_acc: 0.8023

Epoch 00001: val_acc improved from -inf to 0.80226, saving model to inception_v4_difftop.h5
Epoch 2/25
238/237 [==============================] - 559s 2s/step - loss: 0.3026 - acc: 0.8487 - val_loss: 0.3870 - val_acc: 0.8057

Epoch 00002: val_acc improved from 0.80226 to 0.80566, saving model to inception_v4_difftop.h5
Epoch 3/25
238/237 [==============================] - 559s 2s/step - loss: 0.3108 - acc: 0.8443 - val_loss: 0.3376 - val_acc: 0.8293

Epoch 00003: val_acc improved from 0.80566 to 0.82927, saving model to inception_v4_difftop.h5
Epoch 4/25
238/237 [==============================] - 546s 2s/step - loss: 0.3074 - acc: 0.8462 - val_loss: 0.3553 - val_acc: 0.8209

Epoch 00004: val_acc did not improve
Epoch 5/25
238/237 [==============================] - 530s 2s/step - loss: 0.3111 - acc: 0.8430 - val_loss: 0.3659 - val_acc: 0.8154

Epoch 00005

In [8]:
# For test data, rescaling will do
full_test_datagen = ImageDataGenerator(
    rescale = 1./255)
full_test_generator = full_test_datagen.flow_from_directory(
    'data/test',
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    class_mode = "categorical")
res = trained_model.evaluate_generator(full_test_generator)
print(res)

Found 3813 images belonging to 18 classes.
[0.9550693293209311, 0.8400209808706018]


In [26]:
model = load_model("inception_v4_hinge_all.h5")
# For test data, rescaling will do
full_test_datagen = ImageDataGenerator(
    rescale = 1./255)
full_test_generator = full_test_datagen.flow_from_directory(
    'data/test',
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    class_mode = "categorical")
res = model.evaluate_generator(full_test_generator)
print(res)

Found 3813 images belonging to 18 classes.
[0.4735072146023536, 0.8397587201834787]
